In [1]:
import pandas as pd

In [2]:
base_dir = './data/'

data = pd.read_csv(base_dir + 'custom_train.csv')
data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [3]:
# 'class' 값을 기준으로 'label' 컬럼 추가
label_dict = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

data['label'] = data['class'].map(label_dict)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idx           5050 non-null   int64 
 1   class         5050 non-null   object
 2   conversation  5050 non-null   object
 3   label         5050 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 157.9+ KB


In [4]:
# 전처리 함수
# 구분자 삭제
import re

def preprocess_sentence(sentence):
    # 단어와 구두점(punctuation) 사이의 거리를 만듭니다.
    # 예를 들어서 "I am a student." => "I am a student ."와 같이
    # student와 온점 사이에 거리를 만듭니다.
    sentence = sentence.replace("\n", "")         # 구분자
    sentence = sentence.replace("\r", "")         # 구분자
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)

    # (a-z, A-Z,가-힣,0-9, ".", "?", "!", ",")를 제외한 모든 문자를 공백인 ' '로 대체합니다.
    sentence = re.sub(r"[^a-zA-Z가-힣0-9\.\?\!,]"," ",sentence)
    sentence = sentence.strip()
    return sentence

In [5]:
data_prepcocessed = data.copy()
data_prepcocessed['conversation'] = data['conversation'].apply(preprocess_sentence)
data_prepcocessed['conversation'][0]

'지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거면 혼자 죽지 우리까지 사건에 휘말리게 해 ? 진짜 죽여버리고 싶게 . 정말 잘못했습니다 . 너가 선택해 . 너가 죽을래 네 가족을 죽여줄까 . 죄송합니다 . 정말 잘못했습니다 . 너에게는 선택권이 없어 . 선택 못한다면 너와 네 가족까지 모조리 죽여버릴거야 . 선택 못하겠습니다 . 한번만 도와주세요 . 그냥 다 죽여버려야겠군 . 이의 없지 ? 제발 도와주세요 .'

# fasttext txt준비

In [6]:
def add_label(x):
    label = x['label']
    return f'''__label__{label}    {x['conversation']}'''
    

add_label_text = data_prepcocessed.apply(add_label, axis=1)
add_label_text

0       __label__0    지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 ...
1       __label__0    길동경찰서입니다 . 9시 40분 마트에 폭발물을 설치할거다...
2       __label__3    너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . ...
3       __label__1    어이 거기예 ? ? 너 말이야 너 . 이리 오라고무슨 일 ...
4       __label__1    저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선...
                              ...                        
5045    __label__4    10시간만이네요 ! 오늘 데이트는 잘 하셨나요 ? 저는 촬...
5046    __label__4    안녕하세요    저는 30대 여자입니다 . 반갑습니다  저...
5047    __label__4    아이고 역시 사회생활은 힘드네요 ! 면담받고 오니 1시간이...
5048    __label__4    학생들 하교시키고 집에 오니 어느덧 3시간이 자났네요 ! ...
5049    __label__4    배가고파서 그런지 한시만만에 먹었네요    선생님도 고기 ...
Length: 5050, dtype: object

In [7]:
# fasttext 입력으로 주려고 txt 파일 준비
file_path = 'input_fasttext_conversation.txt'

with open(file_path, 'w', encoding='utf8') as f:
    f.write('\n'.join(add_label_text))

# Model

In [8]:
import fasttext

model = fasttext.train_supervised(input=file_path,
                                  epoch=100,
                                  bucket = 20000,
                                  lr = 1,
                                  wordNgrams=2,
                                  dim=100,
                                  )

Read 0M words
Number of words:  72778
Number of labels: 5
Progress: 100.0% words/sec/thread: 1173287 lr:  0.000000 avg.loss:  0.028268 ETA:   0h 0m 0s words/sec/thread: 1139446 lr:  0.900916 avg.loss:  0.270481 ETA:   0h 0m 9s


# test data

In [9]:
# {
#     '협박 대화': 0,
#     '갈취 대화': 1,
#     '직장 내 괴롭힘 대화': 2,
#     '기타 괴롭힘 대화': 3,
#     '일반 대화': 4
# }

In [10]:
test = pd.read_csv('./data/custom_test.csv')

test['conversation'] = test['conversation'].apply(preprocess_sentence)
test.head()

,key,conversation
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나 ? 그럼 영지씨가 가는건 어때 ? 네 ? 제가요...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 . ? 제대로 좀 하지 네 똑바로 좀 하...
3,t_004,아무튼 앞으로 니가 내 와이파이야 . . 응 와이파이 온 . 켰어 . 반말 ? 주인...
4,t_005,그러니까 빨리 말해 . 선생님 제발 살려주십시오 . 비밀번호 틀릴 때마다 손톱 하나...


In [11]:
predictions = model.predict(list(test['conversation']))

# return
# (labels, probabilities)

len(predictions[0])


400

In [12]:
from collections import Counter

results = []
for pred in predictions[0]:
    pred = pred[0].replace('__label__','')
    results.append(pred)

# print(results)
Counter(results)

Counter({'3': 108, '2': 107, '0': 90, '1': 93, '4': 2})

In [13]:
# 'label' 값을 기준으로 'class' 컬럼 추가
class_dict = {
    0: '협박 대화',
    1: '갈취 대화',
    2: '직장 내 괴롭힘 대화',
    3: '기타 괴롭힘 대화',
    4: '일반 대화'
}

In [14]:
label_df = pd.DataFrame(columns=['label'])
test['label'] = results
test['label'] = test['label'].astype(int)
test['class'] = test['label'].map(class_dict)

test.head()


,key,conversation,label,class
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,3,기타 괴롭힘 대화
1,t_001,우리팀에서 다른팀으로 갈 사람 없나 ? 그럼 영지씨가 가는건 어때 ? 네 ? 제가요...,2,직장 내 괴롭힘 대화
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요 . ? 제대로 좀 하지 네 똑바로 좀 하...,2,직장 내 괴롭힘 대화
3,t_004,아무튼 앞으로 니가 내 와이파이야 . . 응 와이파이 온 . 켰어 . 반말 ? 주인...,3,기타 괴롭힘 대화
4,t_005,그러니까 빨리 말해 . 선생님 제발 살려주십시오 . 비밀번호 틀릴 때마다 손톱 하나...,0,협박 대화


In [15]:
samples= test.sample(100)

for text,class_name in zip(samples['conversation'],samples['class']):
    print('-'*30)
    print(text)
    print(class_name)

------------------------------
최주임 오늘 저녁 약속있어 ? ? 박과장님 ! 아니요 없어요 그럼 오늘 둘이서 술한잔 어때 ? 네 ? ? 아니요 . 괜찮습니다 . 집에 가보겠습니다 . 어 왜왜 ! ! 그럼 최주임 집에서 어때 ? 여자가 사는 자취방은 어떤지 궁금하기도 하고 괜찮습니다 . 그래 ? ! ! 오늘 야근할생각하고 ! 최주임 이 프로젝트좀 완성해봐 ! 박과장님 제가 해야할 업무는 다 끝냈고 이건 박과장님 업무인데 아닌가요 ? 선임이 일을 주면 할수도 있지 ! 그래서 못하겠다고 ? 직장의 매운맛을 덜 봤구나 ? 이건 박과장님 제안 거부에 따른 보복조치 라고 느껴지는데요 ? 요즘젊은이는 까탈스럽고 튕기는것도 많아 . 너같은 직원은 같이 일하기 힘든거 아니 ? 말씀이 심하시네요 !
직장 내 괴롭힘 대화
------------------------------
야 돈내놔 없어 10원당 한대 맞고싶냐 ? 진짜 없단말이야 가져오라면 가져와 흑흑 나한테 왜그러는거야 뭐긴 너가 만만하니까 그만 괴롭히면 안돼 ? 이미 많이 뜯어갔잖아 . 나이제 진짜 돈없어 여물고 내일도 잘부탁한다 ?
기타 괴롭힘 대화
------------------------------
야 너 2학년 김민석 맞지 ? 네 맞는데요 . 혹시 누구신가요 ? 내가 누군지 궁금하면 학교 뒤쪽으로 내려와 . 2분 준다 . 네 지금 내려가겠습니다 . 야 니가 김민석이냐 ? 돈 있어 ? 돈 없는데요 . 야 쟤 몸뒤져 . 너 천 원 나올 때마다 얼차려 10분이다 . 야 여기 얘 지갑 있는데 ? 오 지갑만 봐도 돈 많게 생겼네 . 열어봐봐 오 12만원 있네 ? 얼차려가 몇 분이야 ? 야 이 돈 주면 얼차려 없애줄게 . 그거 오늘 저 문제집 사려고 가져온 돈이라 . 혹시 내일 드리면 안될까요 ? 야 니 얼차려 받고 싶냐 ? 그렇게 내일 주고 싶으면 오늘 이 돈도 우리 주고 내일도 돈 가져다 주면 돼 . 뭐가 어려워 . 네 알겠습니다 . 그 12만원 가져게세요 . 말 잘 듣네 . 아주 예뻐 내일도 

# submission

In [22]:
submission = pd.read_csv('./data/submission.csv')
submission['class'] = test['label']

In [23]:
submission.head()

,file_name,class
0,t_000,3
1,t_001,2
2,t_002,2
3,t_004,3
4,t_005,0


In [24]:
submission.to_csv('my_submission.csv', index=False)